### Config

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from chronos_mlx import ChronosPipeline
pd.set_option('display.max_columns', None)

In [5]:
quart = pd.read_parquet('/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/M3 Data/M3_quarter_processed.parquet')

## Loop and Produce Forecasts for All Quarterly Series

In [6]:
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-base",
    dtype="bfloat16",
)

unique_series = quart['Series'].unique()

forecasts = {}

for series_id in unique_series:
    # Filter the dataframe for the current series
    series_data = quart[quart['Series'] == series_id]
    
    # Ensure the data is sorted by date
    series_data = series_data.sort_values('Date')
    
    # Get the number of observations to use for forecasting
    N = series_data['N'].iloc[0]
    NF = series_data['NF'].iloc[0]
    
    # Get the date of the last observation used for training
    last_training_date = series_data['Date'].iloc[N-NF-1]  # -1 because iloc is 0-based

    # Prepare the input data for the model
    training_data = series_data['Value'].iloc[:N-NF].values  # Exclude the last NF values
    
    # Calculate forecast dates assuming quarterly data
    forecast_dates = pd.date_range(start=last_training_date, periods=NF+1, freq='Q')[1:]  # Skip the first since it's the last training date
    
    # Predict the next NF values
    try:
        model_forecasts = pipeline.predict(
            context=training_data,
            prediction_length=NF,
            num_samples=N-NF,
            temperature=1.0,
            top_k=50,
            top_p=1.0
        )#.squeeze(0).squeeze(0).numpy()  # Remove batch and sample dimensions for simplicity
    except Exception as e:
        print(f"Failed to predict for series {series_id} with error: {e}")
        continue
    
    # Quantiles for uncertainty bounds
    low, median, high = np.quantile(model_forecasts[0], [0.1, 0.5, 0.9], axis=0)
    forecasts[series_id] = {
        'Date': forecast_dates,
        'low': low,
        'median': median,
        'high': high
    }

# Convert forecasts dictionary to a DataFrame for easier manipulation
forecast_df = pd.DataFrame()
for series_id, data in forecasts.items():
    df_temp = pd.DataFrame({
        'Date': data['Date'],
        'Series_ID': series_id,
        'Low': data['low'],
        'Median': data['median'],
        'High': data['high']
    })
    forecast_df = pd.concat([forecast_df, df_temp], ignore_index=True)

# forecast_df.set_index(['Date', 'Series_ID'], inplace=True)
print('Forecasting completed for all series.')

Forecasting completed for all series.


In [8]:
forecast_df

,Date,Series_ID,Low,Median,High
0,1993-03-31,646,5411.907185,5512.127688,5645.755026
1,1993-06-30,646,5328.390099,5512.127688,5595.644774
2,1993-09-30,646,5345.093516,5478.720854,5695.865278
3,1993-12-31,646,5261.576430,5478.720854,5729.272112
4,1994-03-31,646,5194.762761,5478.720854,5779.382364
...,...,...,...,...,...
6043,1974-12-31,1401,3489.052786,4321.627566,5683.381232
6044,1975-03-31,1401,2924.595308,4303.988270,5739.826979
6045,1975-06-30,1401,3111.571848,4321.627566,6096.140762
6046,1975-09-30,1401,3425.551320,4815.527859,6392.480938


Slight issue. The datetime values here are a bit different from that in the processed parquet file, still the same quarter, but at the end of the month instead of the start. Should hardly matter. 

Need to decide how to use the low, median and high. Natural to just use median. For now save all of it.

In [9]:
forecast_df.to_parquet('M3_Chronos_quarter_forecast.parquet', index=False,  compression='gzip', engine='pyarrow')